
## DERIVATIVE PRICING
MODULE 1 | LESSON 2


---


# **OPTION PRICING AND PUT-CALL PARITY**

|  |  |
|:---|:---|
|**Reading Time** |  40 minutes |
|**Prior Knowledge** |Financial Derivatives basics, Binomial tree, Risk neutral probabilities  |
|**Keywords** |Risk neutral probabilities, Binomial model, No-arbitrage, Put-Call parity  |


---


*In this notebook, we will continue to develop the binomial model for option pricing in Python. Building on the work we did in Lesson 1, we will now focus on relevant features like the put-call parity. Make sure you understand this properly because it is going to lay the foundation for future work.*


As usual, let's start importing some of the libraries that we may need to use down the road:

In [1]:
import numpy as np

Our first task is going to be to extend the binomial model function used in Lesson 1 to also consider put options:

## 1. Binomial Tree for Put Options


We will create another function, similar to what we did with the call option case, to price put options. The following snippet contains the code for the call option we used in Lesson 1:

In [2]:
def binomial_call_full(S_ini, K, T, r, u, d, N):
    dt = T / N  # Define time step
    p = (np.exp(r * dt) - d) / (u - d)  # Risk neutral probs
    C = np.zeros([N + 1, N + 1])  # Call prices
    S = np.zeros([N + 1, N + 1])  # Underlying price
    for i in range(0, N + 1):
        C[N, i] = max(S_ini * (u ** (i)) * (d ** (N - i)) - K, 0)
        S[N, i] = S_ini * (u ** (i)) * (d ** (N - i))
    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            C[j, i] = np.exp(-r * dt) * (p * C[j + 1, i + 1] + (1 - p) * C[j + 1, i])
            S[j, i] = S_ini * (u ** (i)) * (d ** (j - i))
    return C[0, 0], C, S

Before continuing, can you think of how to modify this accordingly to construct another function that prices put options?

In [3]:
def binomial_put_full(S_ini, K, T, r, u, d, N):
    dt = T / N  # Define time step
    p = (np.exp(r * dt) - d) / (u - d)  # Risk neutral probs
    P = np.zeros([N + 1, N + 1])  # Call prices
    S = np.zeros([N + 1, N + 1])  # Underlying price
    for i in range(0, N + 1):
        P[N, i] = max(K - (S_ini * (u ** (i)) * (d ** (N - i))), 0)
        S[N, i] = S_ini * (u ** (i)) * (d ** (N - i))
    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            P[j, i] = np.exp(-r * dt) * (p * P[j + 1, i + 1] + (1 - p) * P[j + 1, i])
            S[j, i] = S_ini * (u ** (i)) * (d ** (j - i))
    return P[0, 0], P, S

Let's verify this is correct with the numbers from the example in the slides and videos: 

In [4]:
put_price, P, S = binomial_put_full(36, 36, 50, 0.01, 1.1, 0.7, 50)
print("Underlying Price Evolution:\n", S)
print("Put Option Payoff:\n", P)
print("Price at t=0 for Put option is$", "{:.2f}".format(put_price))

Underlying Price Evolution:
 [[3.60000000e+01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.52000000e+01 3.96000000e+01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.76400000e+01 2.77200000e+01 4.35600000e+01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [1.32132126e-06 2.07636197e-06 3.26285453e-06 ... 3.49262042e+03
  0.00000000e+00 0.00000000e+00]
 [9.24924879e-07 1.45345338e-06 2.28399817e-06 ... 2.44483429e+03
  3.84188246e+03 0.00000000e+00]
 [6.47447415e-07 1.01741737e-06 1.59879872e-06 ... 1.71138400e+03
  2.68931772e+03 4.22607070e+03]]
Put Option Payoff:
 [[ 7.5822903   0.          0.         ...  0.          0.
   0.        ]
 [ 9.54336876  7.11166527  0.         ...  0.          0.
   0.        ]
 [11.63290642  9.06090302  6.63837335 ...  0.          0.
   0.        ]
 ...
 [35.28715092 35.28715016 35.28714898 ...  0.          0.
   0.        ]
 [35.64179309 35.64179256 35.64179173 ...  0.         

Next, we will verify whether put-call parity holds in this example at t=0.

## 2. Put-Call Parity in the Binomial Tree

We know that the price of the call and put options with K=90 at t=0 is given by the previous functions:

In [5]:
put_price, P, S = binomial_put_full(100, 90, 10, 0, 1.2, 0.8, 10)
print("Price at t=0 for Put option is $", "{:.2f}".format(put_price))
call_price, C, S = binomial_call_full(100, 90, 10, 0, 1.2, 0.8, 10)
print("Price at t=0 for Call option is $", "{:.2f}".format(call_price))

Price at t=0 for Put option is $ 19.38
Price at t=0 for Call option is $ 29.38


We already know that in order to satisfy absence of arbitrage conditions, the relationship between the price of the put and the call options must be:

$C_0 + Ke^{-rT} = S_0 + P_0$

Let's check whether this is the case for t=0:

In [6]:
round(call_price + 90 * np.exp(-0 * 1), 2) == round(S[0, 0] + put_price, 2)

np.True_

We have verified that the put-call parity here holds perfectly at t=0. The question that arises now is does this behavior also hold at other nodes of the tree (other times)?

In order to check this, let's first increase the number of steps of the binomial tree to N=2:

In [7]:
put_price, P, S = binomial_put_full(100, 90, 1, 0, 1.2, 0.8, 2)
print("Price at t=0 for Put option is $", "{:.2f}".format(put_price))
call_price, C, S = binomial_call_full(100, 90, 1, 0, 1.2, 0.8, 2)
print("Price at t=0 for Call option is $", "{:.2f}".format(call_price))

Price at t=0 for Put option is $ 6.50
Price at t=0 for Call option is $ 16.50


For completeness, let's start verifying Put-Call parity at t=0:

In [8]:
round(call_price + 90 * np.exp(-0 * 1), 2) == round(S[0, 0] + put_price, 2)

np.True_

Now, let's check the same thing for some other node of the two-step tree. Remember that this is the evolution of underlying prices and option payoffs:

In [9]:
print("Underlying Price Evolution:\n", S)
print("Call Option Payoff:\n", C)
print("Put Option Payoff:\n", P)

Underlying Price Evolution:
 [[100.   0.   0.]
 [ 80. 120.   0.]
 [ 64.  96. 144.]]
Call Option Payoff:
 [[16.5  0.   0. ]
 [ 3.  30.   0. ]
 [ 0.   6.  54. ]]
Put Option Payoff:
 [[ 6.5  0.   0. ]
 [13.   0.   0. ]
 [26.   0.   0. ]]


Let's check put-call parity for the node following path 'd' (underlying price S_d = 80), which we can index as [1,0] in the matrix S:

In [10]:
round(C[1, 0] + 90 * np.exp(-0 * 0.5), 2) == round(S[1, 0] + P[1, 0], 2)

np.True_

Now, it is your turn to check whether this relationship holds for other nodes (you can also try trees with more steps) $→$ Does put-call parity hold?

## 3. Conclusion 

In this lesson, we have worked throughout the binomial tree to verify the put-call parity on certain nodes. In the next lesson, we will work on the concept of delta-hedging.

See you there!

---
Copyright 2025 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
